## Multiclassification InceptionV3

In [1]:
import os
from itertools import islice

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import applications

from sklearn.metrics import jaccard_similarity_score

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 299, 299
n_features = 6

train_data_dir = 'multi_class_testing/train/'
validation_data_dir = 'multi_class_testing/test/'

available_train_files = len(os.listdir(train_data_dir + 'female/')) \
    + len(os.listdir(train_data_dir + 'male/'))
available_test_files = len(os.listdir(validation_data_dir + 'female/')) \
    + len(os.listdir(validation_data_dir+'male/'))
    
nb_train_samples  = available_train_files - available_train_files % 8
nb_validation_samples = available_test_files - available_test_files % 8


epochs = 2
batch_size = 8

In [3]:
watch_df = pd.read_csv('all_watch_info_with_indicators.csv')
watch_dict = dict()

In [4]:
for image_name in watch_df['image_name']:
    watch_dict[image_name+'.jpg'] =  watch_df[
        watch_df['image_name'] == image_name][
            ['is_gold','is_silver','is_strap','is_square','is_round','is_male']].values[0]
    

__Custom Generator and Helper Function__

In [23]:
def multiclasses_getter(x, i, gen):
    count = 0
    y = np.zeros((batch_size, n_features), dtype=np.int8)
    
    # Feature not working
    if np.shape(x)[0] < gen.batch_size:
        steps = np.shape(x)[0]
    else:
        steps = gen.batch_size
        
    idx = (i) * gen.batch_size
    
    for f in gen.filenames[idx : idx + steps]:
        k = f.split('/')[-1]
        y[count,:] = watch_dict[k] 
        count += 1
    return y

In [24]:
# def multiclass_flow_from_directory(flow_from_directory_gen, multiclasses_getter):
#     for x, y in flow_from_directory_gen:
#         yield x, multiclasses_getter(x, y)
        
def multiclass_flow_train(flow_from_directory_gen, multiclasses_getter):
    for i, (x, y) in enumerate(flow_from_directory_gen):
        yield x, multiclasses_getter(x, i,train_generator)
    
def multiclass_flow_test(flow_from_directory_gen, multiclasses_getter):
    for i, (x, y) in enumerate(flow_from_directory_gen):
        yield x, multiclasses_getter(x, i, test_generator)
      

In [25]:
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=False)

test_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=False)

Found 7336 images belonging to 2 classes.
Found 3144 images belonging to 2 classes.


In [26]:
watch_df[
        watch_df['image_name'] == 'OmegaDeVillePrestigeCoAxial32742420332005002'][
            ['is_gold','is_silver','is_strap','is_square','is_round','is_male']].values[0]
    

array([1, 1, 0, 0, 1, 0])

In [27]:
model = applications.InceptionV3(include_top=True, weights='imagenet')

In [28]:
layer_name = 'mixed10'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)

In [29]:
watch_model = Sequential()
watch_model.add(intermediate_layer_model)
watch_model.add(GlobalAveragePooling2D(name='avg_pool'))
watch_model.add(Dense(6, activation="sigmoid"))

watch_model.layers[0].trainable = False

In [30]:
watch_model.compile(loss = "binary_crossentropy", optimizer='sgd', metrics=["accuracy"])

In [31]:
watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, None, None, 2048)  21802784  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 12294     
Total params: 21,815,078
Trainable params: 12,294
Non-trainable params: 21,802,784
_________________________________________________________________


In [32]:
# fine-tune the model
watch_model.fit_generator(
        multiclass_flow_train(train_generator, multiclasses_getter),
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = multiclass_flow_test(test_generator, multiclasses_getter),
        validation_steps=nb_validation_samples // batch_size)

Epoch 1/2
917/917 [==============================] - 876s - loss: 0.1779 - acc: 0.9366 - val_loss: 0.3986 - val_acc: 0.9106
Epoch 2/2
917/917 [==============================] - 872s - loss: 0.0426 - acc: 0.9888 - val_loss: 0.0031 - val_acc: 1.0000


In [33]:
watch_model.save('inception_v3_multiclass1.h5')

In [ ]:
# # fine-tune the model
# watch_model.fit_generator(
#         multiclass_flow_from_directory(train_generator, multiclasses_getter),
#         steps_per_epoch=nb_train_samples // batch_size,
#         epochs=epochs,
#         validation_data = multiclass_flow_from_directory(test_generator, multiclasses_getter),
#         validation_steps=nb_validation_samples // batch_size)

In [ ]:
from keras.datasets import cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
def prepare_image(image_path):
    
    im = cv2.imread(image_path)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (299, 299)).astype(np.float32)
    im = im/255
#     im[:,:,0] -= 103.939
#     im[:,:,1] -= 116.779
#     im[:,:,2] -= 123.68
#    im = im.transpose((2,0,1))
    im = np.expand_dims(im, axis=0)
    return im

In [ ]:
pic_path = 'all_pics/'
f_list = os.listdir(pic_path)

X_data = []
for pic_name in f_list:
    next_pic = prepare_image(pic_path + pic_name)
    X_data.append(next_pic)

In [ ]:
X_data = np.array(X_data)

In [ ]:
np.shape(X_data)